This notebook contains all the code necessary to generate the paper Ho, Perley et al. on ZTF18abvkwla (The Koala).

First, import all the packages that we will need.

In [114]:
from penquins import Kowalski
import numpy as np
from astropy.time import Time
import glob
import multiprocessing
import warnings
from helpers import get_lc

# Query for analogs in ZTF

We will do these queries on Kowalski, so we log on there first.

In [7]:
def logon():
    """ Log onto Kowalski """
    username = 'ah'
    password = 'TetraodonInsists'
    s = Kowalski(
        protocol='https', host='kowalski.caltech.edu', port=443,
        verbose=False, username=username, password=password)
    return s

### Selection of field-nights

First I need to decide which field-nights to include in the search. We use the following criteria:

* At least one observation the night before (0.5 < dt < 1.5 days)
* At least one observation two nights before (2.5 < dt < 1.5 days)
* At least three observations in the next five nights (dt < 5.5 days)

In [33]:
def load():
    ut = []
    field = []
    filt = []
    ddir = "/Users/annaho/Dropbox/Projects/Research/Koala/data"
    for line in open("%s/allexp.tbl" %ddir, "r"):
        # not a header line
        if '1DC' in line:
            dat = line.split(' ')
            dat = [i for i in dat if i]
            ut.append(dat[0])
            field.append(dat[5])
            filt.append(dat[3])
    ut = np.array(ut)
    field = np.array(field)
    filt = np.array(filt)
    return ut,field,filt

In [39]:
def calc_n_active_nights():
    ut, field, filt = load()

    # Simplify the UT thing to just the date
    ut_day = np.array([i.split('T')[0] for i in ut])

    # Unique fields
    ufields = np.unique(field)

    # Number of active nights for that field
    nnights = np.zeros(len(ufields))

    # For each field, count the number of nights with 1 obs
    # AND an obs the night before AND an obs the night before that
    # AND 3 obs in the next 5 nights
    outputf = open("../data/fieldnights_1DC.txt", "w")
    for ii,uf in enumerate(ufields):
        print(uf)
        n_active_nights = 0
        field_day = Time(ut_day[field==uf], format='isot')
        unique_field_day = np.unique(field_day)
        for day in unique_field_day:
            nobs = sum(field_day==day) >= 1 # number of points in the night
            if nobs:
                dt = day-unique_field_day
                nobs_before = sum(np.logical_and(dt < 1.5, dt > 0.5)) >= 1
                nobs_2_before = sum(np.logical_and(dt < 2.5, dt > 1.5)) >= 1
                nobs_after = sum(np.logical_and(dt > -5.5, dt < -0.5)) >= 3
                if np.logical_and(nobs_before, nobs_after):
                    n_active_nights += 1
                    print(uf, str(day).split('T')[0])
                    outputf.write("%s,%s" %(uf, str(day).split('T')[0]))
                    outputf.write('\n')
        print(n_active_nights)
        nnights[ii] = n_active_nights
    outputf.close()

    # Save as array
    np.savetxt("../data/n_active_nights.txt",
            np.array([ufields,nnights]).T, fmt='%s',
            delimiter=',')

In [42]:
calc_n_active_nights()

245
245 2018-08-13
245 2018-08-14
245 2018-08-15
245 2018-08-18
245 2018-08-19
245 2018-08-30
245 2018-08-31
245 2018-09-01
245 2018-09-02
245 2018-09-03
245 2018-09-04
245 2018-09-05
245 2018-09-06
245 2018-09-07
245 2018-09-08
245 2018-09-09
245 2018-09-10
245 2018-09-11
245 2018-09-12
245 2018-09-13
245 2018-09-14
245 2018-09-15
245 2018-09-16
245 2018-09-17
245 2018-09-18
245 2018-09-19
245 2018-09-22
245 2018-09-23
245 2018-09-26
29
246
246 2018-08-14
246 2018-08-18
246 2018-08-19
246 2018-08-20
246 2018-08-24
246 2018-08-25
246 2018-08-26
246 2018-08-27
246 2018-08-28
246 2018-08-29
246 2018-08-30
246 2018-08-31
246 2018-09-01
246 2018-09-02
246 2018-09-03
246 2018-09-04
246 2018-09-05
246 2018-09-06
246 2018-09-09
246 2018-09-10
246 2018-09-11
246 2018-09-12
246 2018-09-13
246 2018-09-16
246 2018-09-17
246 2018-09-22
246 2018-09-23
246 2018-09-24
246 2018-09-25
246 2018-09-26
30
247
247 2018-08-25
247 2018-08-26
247 2018-08-27
247 2018-08-28
247 2018-08-29
247 2018-08-30
247 201

302 2018-09-22
302 2018-09-23
302 2018-09-26
302 2019-01-04
302 2019-01-05
302 2019-01-08
302 2019-01-23
302 2019-01-24
27
303
303 2019-01-04
303 2019-01-05
303 2019-01-08
303 2019-01-25
4
304
304 2018-11-01
304 2018-11-02
304 2018-11-03
304 2018-11-04
304 2018-11-05
304 2018-11-06
304 2018-11-07
304 2018-11-08
304 2018-11-09
304 2018-11-10
304 2019-01-04
304 2019-01-05
304 2019-01-08
13
305
305 2018-11-01
305 2018-11-02
305 2018-11-03
305 2018-11-04
305 2018-11-05
305 2018-11-06
305 2018-11-07
305 2018-11-08
305 2018-11-09
305 2018-11-10
305 2019-01-05
305 2019-01-08
305 2019-01-23
305 2019-01-24
305 2019-01-25
15
306
306 2018-11-01
306 2018-11-02
306 2018-11-03
306 2018-11-04
306 2018-11-05
306 2018-11-08
306 2018-11-09
306 2018-11-10
306 2018-11-13
306 2018-11-14
306 2018-11-15
306 2018-11-16
12
307
307 2018-11-01
307 2018-11-04
307 2018-11-05
307 2018-11-10
307 2018-11-13
307 2018-11-14
307 2018-11-15
307 2018-11-16
307 2019-03-15
9
313
0
314
314 2018-12-17
314 2019-01-23
314 2019-

363 2018-12-15
363 2018-12-16
363 2018-12-17
15
364
364 2018-11-01
364 2018-11-04
364 2018-11-05
364 2018-11-08
364 2018-11-09
364 2018-11-10
364 2018-11-15
364 2018-11-16
364 2018-12-14
364 2018-12-15
10
365
365 2018-12-14
365 2018-12-15
2
366
366 2018-12-13
366 2018-12-14
366 2018-12-15
3
367
367 2018-12-13
367 2018-12-14
367 2018-12-15
3
368
368 2018-12-14
368 2018-12-15
2
374
0
388
388 2018-11-02
388 2018-11-03
388 2018-11-04
388 2018-11-05
388 2019-07-29
388 2019-07-30
388 2019-07-31
388 2019-08-03
388 2019-08-17
9
389
389 2018-11-03
389 2018-11-04
389 2018-11-05
389 2018-11-06
389 2018-11-07
389 2018-11-08
389 2019-07-27
389 2019-07-28
389 2019-07-29
389 2019-08-01
389 2019-08-02
389 2019-08-03
389 2019-08-04
13
392
392 2018-11-02
392 2018-11-03
392 2018-11-06
392 2018-11-07
392 2018-11-08
392 2018-11-09
392 2018-11-10
392 2018-11-11
392 2019-07-27
392 2019-07-28
392 2019-07-29
392 2019-07-30
392 2019-07-31
392 2019-08-01
392 2019-08-02
392 2019-08-03
392 2019-08-04
392 2019-08-0

8
418
0
425
425 2018-12-17
1
426
426 2019-01-20
1
427
427 2019-01-20
427 2019-04-09
427 2019-04-10
427 2019-04-11
427 2019-07-15
427 2019-07-16
427 2019-07-17
427 2019-07-18
427 2019-07-19
427 2019-07-20
427 2019-07-21
11
439
439 2018-11-04
439 2018-11-05
439 2018-11-06
439 2019-07-27
439 2019-07-28
439 2019-07-29
439 2019-07-30
439 2019-07-31
439 2019-08-03
439 2019-08-06
439 2019-08-17
11
443
443 2018-11-02
443 2018-11-03
443 2018-11-04
443 2018-11-05
443 2018-11-06
443 2018-11-07
443 2018-11-08
443 2018-11-09
443 2018-11-10
443 2018-11-11
443 2019-07-14
443 2019-07-27
443 2019-07-28
443 2019-07-29
443 2019-07-30
443 2019-07-31
443 2019-08-01
443 2019-08-02
443 2019-08-03
443 2019-08-04
443 2019-08-05
443 2019-08-06
443 2019-08-07
443 2019-08-08
24
448
448 2018-08-16
448 2018-08-17
448 2018-08-18
448 2018-08-19
448 2018-08-20
448 2018-08-21
448 2018-08-22
448 2018-08-23
448 2018-08-24
448 2018-09-01
448 2018-09-02
448 2018-09-03
448 2018-09-04
448 2018-09-05
448 2018-09-06
448 2018-0

498 2019-08-04
498 2019-08-05
498 2019-08-06
498 2019-08-09
498 2019-08-10
498 2019-08-13
55
499
499 2018-08-16
499 2018-08-17
499 2018-08-18
499 2018-08-19
499 2018-08-20
499 2018-08-21
499 2018-08-22
499 2018-08-23
499 2018-08-24
499 2018-08-25
499 2018-08-26
499 2018-08-27
499 2018-08-31
499 2018-09-01
499 2018-09-02
499 2018-09-03
499 2018-09-04
499 2018-09-05
499 2018-09-06
499 2018-09-07
499 2018-09-08
499 2018-09-09
499 2018-09-10
499 2018-09-11
499 2018-09-12
499 2018-09-13
499 2018-09-14
499 2018-09-15
499 2018-09-16
499 2018-09-17
499 2018-09-18
499 2018-09-19
499 2018-09-22
499 2018-09-23
499 2018-09-24
499 2018-11-01
499 2018-11-02
499 2018-11-03
499 2018-11-04
499 2018-11-05
499 2018-11-06
499 2018-11-07
499 2018-11-08
499 2018-11-09
499 2018-11-10
499 2018-11-13
499 2019-01-05
499 2019-01-08
499 2019-07-14
499 2019-07-15
499 2019-07-16
499 2019-07-17
499 2019-07-18
499 2019-07-19
499 2019-07-27
499 2019-07-30
499 2019-07-31
499 2019-08-01
499 2019-08-02
499 2019-08-03
499

...

718 2018-04-22
718 2018-04-23
718 2018-04-24
718 2018-04-25
718 2018-04-26
718 2018-05-05
718 2018-05-06
718 2018-05-07
718 2018-05-08
718 2018-05-09
16
719
719 2018-04-19
719 2018-04-22
719 2018-04-23
719 2018-04-24
719 2018-04-25
719 2018-04-26
719 2018-05-05
719 2018-05-06
719 2018-05-07
719 2018-05-08
719 2018-05-18
11
720
720 2018-04-19
720 2018-04-22
720 2018-04-23
720 2018-04-24
720 2018-04-25
720 2018-04-26
720 2018-05-05
720 2018-05-06
720 2018-05-07
720 2018-05-08
720 2018-05-18
11
721
721 2018-04-09
721 2018-04-19
721 2018-04-22
721 2018-04-23
721 2018-04-24
721 2018-04-25
721 2018-04-26
721 2018-05-05
721 2018-05-06
721 2018-05-07
721 2018-05-08
721 2018-05-17
12
722
722 2018-04-09
722 2018-04-19
722 2018-04-22
722 2018-04-23
722 2018-04-24
722 2018-04-25
722 2018-04-26
722 2018-05-05
722 2018-05-06
722 2018-05-07
722 2018-05-08
722 2018-05-17
12
727
727 2018-08-11
1
728
728 2018-08-11
1
731
0
732
732 2019-09-07
1
733
733 2019-09-07
733 2019-09-08
2
734
734 2018-08-13
734 2

752 2018-04-16
752 2018-04-26
752 2018-04-27
752 2018-05-05
752 2018-05-06
752 2018-05-07
752 2018-05-08
752 2018-05-09
752 2018-05-20
752 2018-05-29
752 2018-06-09
752 2018-06-10
752 2018-06-11
752 2018-11-01
752 2018-11-02
752 2018-11-03
752 2018-11-04
752 2018-11-05
752 2018-11-06
752 2018-11-07
752 2018-11-08
752 2018-11-09
752 2018-11-10
752 2018-11-13
752 2018-11-14
752 2018-11-15
752 2018-11-16
752 2018-12-10
752 2018-12-13
752 2018-12-14
752 2018-12-15
752 2018-12-16
752 2018-12-17
752 2019-01-20
752 2019-01-23
752 2019-01-24
752 2019-01-25
752 2019-01-26
752 2019-01-27
752 2019-01-28
44
753
753 2018-04-08
753 2018-04-09
753 2018-04-19
753 2018-04-23
753 2018-04-24
753 2018-04-25
753 2018-04-26
753 2018-04-27
753 2018-05-05
753 2018-05-06
753 2018-05-07
753 2018-05-08
753 2018-05-09
753 2018-05-17
753 2018-05-20
753 2018-05-29
753 2018-05-30
753 2018-05-31
753 2018-06-03
753 2018-06-08
753 2018-06-09
753 2018-06-12
753 2018-11-01
753 2018-11-02
753 2018-11-03
753 2018-11-04
753

786 2019-01-28
786 2019-02-25
786 2019-03-16
786 2019-03-17
786 2019-04-08
786 2019-04-09
786 2019-04-12
786 2019-04-13
786 2019-04-14
786 2019-04-15
786 2019-04-16
786 2019-04-19
786 2019-04-24
786 2019-04-25
786 2019-04-26
786 2019-04-27
786 2019-04-28
53
787
787 2018-04-07
787 2018-04-08
787 2018-04-09
787 2018-04-10
787 2018-04-11
787 2018-04-22
787 2018-04-23
787 2018-04-24
787 2018-04-25
787 2018-04-26
787 2018-04-27
787 2018-05-05
787 2018-05-06
787 2018-05-07
787 2018-05-08
787 2018-05-09
787 2018-05-20
787 2018-05-29
787 2018-05-30
787 2018-05-31
787 2018-06-03
787 2018-06-08
787 2018-06-09
787 2018-06-10
787 2018-06-11
787 2018-06-12
787 2018-11-01
787 2018-11-02
787 2018-11-03
787 2018-11-04
787 2018-11-05
787 2018-11-06
787 2018-11-07
787 2018-11-08
787 2018-11-09
787 2018-11-10
787 2018-11-15
787 2018-11-16
787 2018-12-10
787 2018-12-13
787 2018-12-14
787 2018-12-15
787 2018-12-16
787 2018-12-17
787 2019-01-20
787 2019-01-23
787 2019-01-24
787 2019-01-25
787 2019-01-26
787

814 2018-11-03
814 2018-11-04
814 2018-11-05
814 2018-11-06
814 2018-11-07
814 2018-11-08
814 2018-11-09
814 2018-11-10
814 2018-11-13
814 2018-11-14
814 2018-11-15
814 2018-11-16
814 2018-12-10
814 2018-12-13
814 2018-12-14
814 2018-12-15
814 2018-12-16
814 2018-12-17
814 2018-12-20
814 2019-02-25
814 2019-03-15
814 2019-03-16
814 2019-03-17
814 2019-04-09
814 2019-04-12
814 2019-04-13
814 2019-04-14
814 2019-04-15
814 2019-04-16
59
815
815 2018-11-01
815 2018-11-02
815 2018-11-03
815 2018-11-04
815 2018-11-05
815 2018-11-06
815 2018-11-07
815 2018-11-08
815 2018-11-09
815 2018-11-10
815 2018-11-13
815 2018-11-14
815 2018-11-15
815 2018-11-16
815 2018-12-10
815 2018-12-13
815 2018-12-14
815 2018-12-15
815 2018-12-16
815 2018-12-17
815 2019-01-04
815 2019-01-08
815 2019-01-21
815 2019-01-22
815 2019-01-23
815 2019-01-24
815 2019-01-25
815 2019-01-26
815 2019-01-27
815 2019-01-28
815 2019-02-25
815 2019-03-15
815 2019-03-16
815 2019-03-17
815 2019-04-09
815 2019-04-12
815 2019-04-13
815

838 2018-12-15
838 2018-12-16
838 2018-12-17
838 2018-12-20
838 2019-03-15
838 2019-03-16
838 2019-03-17
838 2019-04-09
838 2019-04-12
838 2019-04-13
838 2019-04-14
67
839
839 2018-08-26
839 2018-08-27
839 2018-08-28
839 2018-08-29
839 2018-08-30
839 2018-08-31
839 2018-09-01
839 2018-09-02
839 2018-09-03
839 2018-09-04
839 2018-09-05
839 2018-09-06
839 2018-09-07
839 2018-09-08
839 2018-09-09
839 2018-09-10
839 2018-09-11
839 2018-09-12
839 2018-09-13
839 2018-09-14
839 2018-09-15
839 2018-09-16
839 2018-09-17
839 2018-09-18
839 2018-09-19
839 2018-09-22
839 2018-09-23
839 2018-09-24
839 2018-09-25
839 2018-09-26
839 2018-11-01
839 2018-11-02
839 2018-11-03
839 2018-11-04
839 2018-11-05
839 2018-11-06
839 2018-11-07
839 2018-11-08
839 2018-11-09
839 2018-11-10
839 2018-11-13
839 2018-11-14
839 2018-11-15
839 2018-11-16
839 2018-12-13
839 2018-12-14
839 2018-12-15
839 2018-12-16
839 2018-12-17
839 2018-12-20
839 2019-03-15
839 2019-03-16
839 2019-03-17
839 2019-04-09
839 2019-04-12
839

In [47]:
dat = np.loadtxt("../data/n_active_nights.txt", delimiter=',')
sum(dat[:,1])

8064.0

The total is 8064 field-days, and we save the field-nights to a file in the Koala/data directory. We will use this file to run our queries.

For each field-night, we will do a very loose query for all sources that are real (drb > 0.99) and not stars. We will save that list to a file, and use this file to perform a more detailed query later.

In [73]:
def get_query(start,end,field):
    q = {"query_type": "find",
         "query": {
             "catalog": "ZTF_alerts",
             "filter": {
                     'candidate.jd': {'$gt': start, '$lt': end},
                     'candidate.field': {'$eq': field},
                     'classifications.braai': {'$gt': 0.99},
                     'candidate.magpsf': {'$lt': 20},
                     'candidate.isdiffpos': {'$in': ['1', 't']},
             },
             "projection": {
                     "objectId": 1,
                     "candidate.ra": 1,
                     "candidate.dec": 1,
                     "candidate.distpsnr1": 1,
                     "candidate.sgscore1": 1,
             },
         "kwargs": {
             "hint": "jd_field_rb_drb_braai_ndethhist_magpsf_isdiffpos",
         }
         }
         }
    return q

In [96]:
def search(s, field, start, end):
    q = get_query(start,end,field)
    r = s.query(query=q)
    out = r['result_data']['query_result']
    
    # Get rid of stars 
    distnr = np.array([d['candidate']['distpsnr1'] for d in out])
    sgscore = np.array([d['candidate']['sgscore1'] for d in out])
    star = np.logical_and(distnr<1.0, sgscore>0.76)
    oids = np.array([d['objectId'] for d in out])[~star]
    
    # Return unique candidates
    oids_unique, inds = np.unique(oids, return_index=True)
    ras = np.array([d['candidate']['ra'] for d in out])[~star][inds]
    decs = np.array([d['candidate']['dec'] for d in out])[~star][inds]

    return oids_unique

In [56]:
class Day:
    """ Get a date as a datestr, as a time object, and get to the next day """
    def __init__(self, date):
        self.timeobj = Time(date, format='isot')
        self.datestr = self.timeobj.iso[0:10].replace('-','')

    def next_day(self):
        timeobj = Time(self.timeobj.jd + 1, format='jd')
        day2 = Day(timeobj.iso[0:10])
        return day2

In [97]:
dat = np.loadtxt("../data/fieldnights_1DC.txt", dtype=str, delimiter=',')
fld = dat[:,0].astype(int)
date = [Day(val) for val in dat[:,1]]

In [130]:
for ii,fld_num in enumerate(fld):
    outf = "output/%s_%s.txt" %(date[ii].datestr,fld_num)

    if len(glob.glob(outf)) == 0:
        print(100*ii/len(fld))
        #print("Running for %s,%s" %(date[ii].datestr,fld_num))
        start = date[ii].timeobj.jd
        end = start+1
        s = logon()
        oids = search(s,int(fld_num),start,end)

        #print(oids)

        np.savetxt(outf, oids, fmt='%s')

92.27430555555556
92.28670634920636
92.29910714285714
92.31150793650794
92.32390873015873
92.33630952380952
92.34871031746032
92.36111111111111
92.3735119047619
92.3859126984127
92.3983134920635
92.41071428571429
92.42311507936508
92.43551587301587
92.44791666666667
92.46031746031746
92.47271825396825
92.48511904761905
92.49751984126983
92.50992063492063
92.52232142857143
92.53472222222223
92.54712301587301
92.55952380952381
92.57192460317461
92.58432539682539
92.59672619047619
92.60912698412699
92.62152777777777
92.63392857142857
92.64632936507937
92.65873015873017
92.67113095238095
92.68353174603175
92.69593253968254
92.70833333333333
92.72073412698413
92.73313492063492
92.74553571428571
92.7579365079365
92.7703373015873
92.7827380952381
92.79513888888889
92.80753968253968
92.81994047619048
92.83234126984127
92.84474206349206
92.85714285714286
92.86954365079364
92.88194444444444
92.89434523809524
92.90674603174604
92.91914682539682
92.93154761904762
92.94394841269842
92.9563492063492

98.01587301587301
98.02827380952381
98.04067460317461
98.05307539682539
98.06547619047619
98.07787698412699
98.09027777777777
98.10267857142857
98.11507936507937
98.12748015873017
98.13988095238095
98.15228174603175
98.16468253968254
98.17708333333333
98.18948412698413
98.20188492063492
98.21428571428571
98.2266865079365
98.2390873015873
98.2514880952381
98.26388888888889
98.27628968253968
98.28869047619048
98.30109126984127
98.31349206349206
98.32589285714286
98.33829365079364
98.35069444444444
98.36309523809524
98.37549603174604
98.38789682539682
98.40029761904762
98.41269841269842
98.4250992063492
98.4375
98.4499007936508
98.46230158730158
98.47470238095238
98.48710317460318
98.49950396825396
98.51190476190476
98.52430555555556
98.53670634920636
98.54910714285714
98.56150793650794
98.57390873015873
98.58630952380952
98.59871031746032
98.61111111111111
98.6235119047619
98.6359126984127
98.6483134920635
98.66071428571429
98.67311507936508
98.68551587301587
98.69791666666667
98.7103174

Let's calculate the number of sources that passed this initial broad query.

In [133]:
files = glob.glob("output/*.txt")
src = []
for f in files:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        [src.append(val) for val in np.loadtxt(f, dtype=str, ndmin=2)]
src = np.unique(np.array(src))

The total number of sources that passed this initial broad query is 758,528.

For each of these sources, let's require a certain LC quality and duration.

In [ ]:
files = glob.glob("output/*.txt")
#files = ["output/20180914_554.txt"]
for ii,f in enumerate(files):
    print(ii/len(files))
    keep = []
    string = f.split(".")[0]
    newf = string + "_duration.txt"
    if len(glob.glob(newf))==0:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            sources = np.unique(np.loadtxt(f,dtype=str))
            for source in sources:
                isalert,jd,mag,emag,filt,program,limjds,limmags,limfilts = get_lc(s, source)
                # For each filter individually, 
                # count the number of detections and the duration
                choose = np.logical_and.reduce((filt == 1, program==3, mag<99))
                ndet = sum(choose)
                if ndet >= 3:
                    dur = max(jd[choose])-min(jd[choose])
                    if np.logical_and(dur>1, dur<100):
                        print(source)
                        jd_max = jd[choose][np.argmin(mag[choose])]
                        mag_max = np.min(mag[choose])
                        # Preceding measurements
                        preceding = np.logical_and(jd>jd_max-5,jd<jd_max)
                        has_det = sum(mag[preceding] >= mag_max+0.75)>0
                        preceding = np.logical_and(limjds>jd_max-5,limjds<jd_max)
                        has_nondet = sum(limmags[preceding] >= mag_max+0.75)>0
                        # Subsequent measurements
                        if np.logical_or(has_det,has_nondet):
                            subsequent = np.logical_and(jd<jd_max+5,jd>jd_max)
                            has_det = sum(mag[subsequent] >= mag_max+0.75)>0
                            subsequent = np.logical_and(limjds>jd_max-5,limjds<jd_max)
                            has_nondet = sum(limmags[subsequent] >= mag_max+0.75)>0   
                            if np.logical_or(has_det,has_nondet):
                                print("LC good")
                                keep.append(source)  

                choose = np.logical_and.reduce((filt == 2, program==3, mag<99))
                ndet = sum(choose)
                if ndet >= 3:
                    dur = max(jd[choose])-min(jd[choose])
                    if np.logical_and(dur>1, dur<100):
                        print(source)
                        jd_max = jd[choose][np.argmin(mag[choose])]
                        mag_max = np.min(mag[choose])
                        # Preceding measurements
                        preceding = np.logical_and(jd>jd_max-5,jd<jd_max)
                        has_det = sum(mag[preceding] >= mag_max+0.75)>0
                        preceding = np.logical_and(limjds>jd_max-5,limjds<jd_max)
                        has_nondet = sum(limmags[preceding] >= mag_max+0.75)>0
                        # Subsequent measurements
                        if np.logical_or(has_det,has_nondet):
                            subsequent = np.logical_and(jd<jd_max+5,jd>jd_max)
                            has_det = sum(mag[subsequent] >= mag_max+0.75)>0
                            subsequent = np.logical_and(limjds>jd_max-5,limjds<jd_max)
                            has_nondet = sum(limmags[subsequent] >= mag_max+0.75)>0   
                            if np.logical_or(has_det,has_nondet):
                                print("LC good")
                                keep.append(source) 
    np.savetxt("%s" %newf,keep,fmt='%s')

0.0
0.00011763321962122103
0.00023526643924244207
0.0003528996588636631
0.00047053287848488413
0.0005881660981061052
0.0007057993177273262
0.0008234325373485472
0.0009410657569697683
0.0010586989765909893
0.0011763321962122103
0.0012939654158334313
0.0014115986354546525
0.0015292318550758734
0.0016468650746970944
0.0017644982943183156
0.0018821315139395365
0.0019997647335607575
0.0021173979531819787
0.0022350311728031994
0.0023526643924244206
0.002470297612045642
0.0025879308316668626
0.0027055640512880837
0.002823197270909305
0.0029408304905305257
0.003058463710151747
0.003176096929772968
0.0032937301493941888
0.00341136336901541
0.003528996588636631
0.003646629808257852
0.003764263027879073
0.0038818962475002943
0.003999529467121515
0.004117162686742736
0.004234795906363957
0.0043524291259851786
0.004470062345606399
0.00458769556522762
0.004705328784848841
0.004822962004470062
0.004940595224091284
0.005058228443712505
0.005175861663333725
0.005293494882954946
0.0054111281025761675
0.

0.05564051288083755
0.05575814610045877
0.05587577932007999
0.055993412539701214
0.05611104575932243
0.05622867897894365
0.056346312198564875
0.0564639454181861
0.056581578637807314
0.05669921185742854
0.05681684507704976
0.05693447829667098
0.0570521115162922
0.05716974473591342
0.057287377955534645
0.05740501117515586
0.057522644394777084
0.05764027761439831
0.05775791083401953
0.057875544053640746
0.05799317727326197
0.05811081049288319
0.058228443712504414
0.05834607693212563
0.05846371015174685
0.058581343371368076
0.05869897659098929
0.058816609810610515
0.05893424303023174
0.05905187624985296
0.05916950946947418
0.0592871426890954
0.05940477590871662
0.059522409128337846
0.05964004234795906
0.059757675567580285
0.05987530878720151
0.05999294200682272
0.060110575226443946
0.06022820844606517
0.06034584166568639
0.06046347488530761
0.06058110810492883
0.060698741324550054
0.06081637454417128
0.06093400776379249
0.061051640983413716
0.06116927420303494
0.061286907422656155
0.061404

0.08140218797788495
0.08151982119750617
ZTF18aakoylt
LC good
ZTF18aaobdza
ZTF18aaobguk
ZTF18aaobguk
LC good
0.0816374544171274
0.08175508763674862
ZTF18aaajxfr
LC good
ZTF18aaajxfr
LC good
ZTF18aagrtxs
LC good
ZTF18aagrtxs
LC good
ZTF18aahiqfi
LC good
ZTF18aahiqfi
LC good
ZTF18aahjdxh
LC good
ZTF18aahjdxh
LC good
0.08187272085636983
ZTF17aabwoss
LC good
ZTF17aabwoss
LC good
ZTF18aagrded
LC good
ZTF18aahjfie
ZTF18aahjfie
ZTF18aakrheg
ZTF18aakrheg
0.08199035407599106
ZTF18aabemwy
LC good
ZTF18aacckuy
LC good
ZTF18aagrcir
LC good
ZTF18aagrded
LC good
ZTF18aaguhgb
LC good
ZTF18aakqibg
ZTF18aakvers
LC good
ZTF18aakvers
LC good
0.08210798729561228
ZTF18aaguhgb
LC good
0.08222562051523351
ZTF18aakwwjx
0.08234325373485472
0.08246088695447594
ZTF18aaobdgz
LC good
0.08257852017409717
ZTF18aairuad
LC good
ZTF18aairuad
LC good
0.08269615339371839
0.0828137866133396
ZTF18aajqcue
LC good
ZTF18aajqcue
LC good
ZTF18aakecej
LC good
ZTF18aakecej
LC good
ZTF18aakqldk
LC good
ZTF18aakqldk
LC good
0.082931

ZTF18aaisyyp
LC good
ZTF18aaisyyp
LC good
0.10045876955652276
0.10057640277614398
0.10069403599576521
0.10081166921538642
0.10092930243500764
0.10104693565462887
0.10116456887425009
0.10128220209387132
0.10139983531349253
0.10151746853311375
0.10163510175273498
0.10175273497235619
0.10187036819197741
0.10198800141159864
0.10210563463121985
0.10222326785084107
0.1023409010704623
0.10245853429008352
0.10257616750970475
0.10269380072932596
0.10281143394894718
ZTF18aakrheg
ZTF18aakrheg
0.10292906716856841
0.10304670038818962
0.10316433360781084
0.10328196682743207
ZTF18aairuad
LC good
ZTF18aairuad
LC good
ZTF18aaobguk
ZTF18aaobguk
LC good
0.10339960004705329
ZTF18aakecej
LC good
ZTF18aakecej
LC good
0.1035172332666745
0.10363486648629573
0.10375249970591695
0.10387013292553818
0.1039877661451594
0.10410539936478061
ZTF18aaisdey
LC good
ZTF18aaisdey
LC good
ZTF18aaisyyp
LC good
ZTF18aaisyyp
LC good
ZTF18aakdyss
LC good
ZTF18aakdyss
0.10422303258440184
ZTF18aagtevz
LC good
ZTF18aagtevz
LC go

ZTF18aaohapd
LC good
ZTF18aaqehoc
LC good
ZTF18aaqehoc
0.1141042230325844
ZTF18aajqlkp
ZTF18aamdkki
LC good
ZTF18aaqfffw
ZTF18aaqfffw
0.11422185625220563
ZTF18aalbpll
LC good
0.11433948947182684
ZTF18aaqqhis
ZTF18aaqqhis
0.11445712269144806
ZTF18aaqkoyr
LC good
ZTF18aaqkoyr
0.11457475591106929
ZTF18aaobguk
ZTF18aaobguk
LC good
ZTF18aaqdvcs
0.1146923891306905
ZTF18aapiqgt
ZTF18aaqeasu
0.11481002235031172
ZTF18aabldbr
LC good
ZTF18aabldbr
LC good
ZTF18aagrtxs
LC good
ZTF18aagrtxs
LC good
ZTF18aahweiu
LC good
ZTF18aambmoh
ZTF18aambmoh
LC good
ZTF18aaqcqkv
ZTF18aaqcqkv
ZTF18aaqcqve
LC good
ZTF18aaqcqve
LC good
ZTF18aaqcqvr
LC good
ZTF18aaqcqvr
0.11492765556993295
ZTF17aabxoet
LC good
ZTF17aabxoet
LC good
ZTF18aabemhv
LC good
ZTF18aagrded
LC good
ZTF18aaogeaz
ZTF18aaogeaz
ZTF18aaqctmx
LC good
ZTF18aaqctmx
LC good
ZTF18aaqjxid
LC good
ZTF18aaqjxid
LC good
0.11504528878955417
ZTF18aabemwy
LC good
ZTF18aagrcir
LC good
ZTF18aagrded
LC good
ZTF18aaguhgb
LC good
ZTF18aaizewi
LC good
ZTF18aapsedq


ZTF18aahiqfi
LC good
ZTF18aahiqfi
LC good
ZTF18aambmoh
ZTF18aambmoh
LC good
ZTF18aaqcqkv
ZTF18aaqcqkv
ZTF18aaqcqve
LC good
ZTF18aaqcqve
LC good
ZTF18aaqcqvr
LC good
ZTF18aaqcqvr
0.1275144100694036
ZTF17aabxoet
LC good
ZTF17aabxoet
LC good
ZTF18aabemhv
LC good
ZTF18aagrded
LC good
ZTF18aakebhi
LC good
ZTF18aakebhi
LC good
ZTF18aaogeaz
ZTF18aaogeaz
ZTF18aaqjxid
LC good
ZTF18aaqjxid
LC good
0.12763204328902483
ZTF18aacckuy
LC good
ZTF18aacckvv
LC good
ZTF18aaizewi
LC good
ZTF18aapsedq
LC good
ZTF18aapsedq
LC good
ZTF18aaqcxzf
ZTF18aaqczgk
LC good
ZTF18aaqczgk
LC good
0.12774967650864605
ZTF18aaizewi
LC good
ZTF18aalrqbo
LC good
ZTF18aapbpde
ZTF18aapbpde
ZTF18aaqfchn
LC good
ZTF18aaqfchn
LC good
0.12786730972826726
ZTF18aaineku
LC good
ZTF18aaiyzuz
LC good
ZTF18aalrqbo
LC good
ZTF18aaqmaet
LC good
0.12798494294788848
0.1281025761675097
0.1282202093871309
0.12833784260675216
0.12845547582637337
0.1285731090459946
ZTF18aajqcue
LC good
ZTF18aajqcue
LC good
ZTF18aakecej
LC good
ZTF18aakecej
LC

ZTF18aaqaqyn
LC good
ZTF18aaqaqyn
LC good
ZTF18aaqmvba
LC good
ZTF18aasdzhy
LC good
ZTF18aauaogp
ZTF18aauaogp
LC good
0.1376308669568286
ZTF18aaqeygf
ZTF18aaqeygf
ZTF18aauhqdo
ZTF18aauhqdo
0.13774850017644982
ZTF18aapictz
ZTF18aaqjovh
LC good
ZTF18aauiqkr
LC good
0.13786613339607104
ZTF18aaqeszk
0.13798376661569228
ZTF18aahjafd
LC good
ZTF18aahjbii
0.1381013998353135
0.13821903305493471
ZTF18aahhzqn
LC good
ZTF18aahrzrb
LC good
0.13833666627455593
ZTF18aaisknu
LC good
ZTF18aaqdozs
ZTF18aaqdozs
ZTF18aatfboz
LC good
0.13845429949417715
ZTF18aaqptgz
ZTF18aaqptgz
LC good
ZTF18aardcky
ZTF18aardcky
LC good
ZTF18aarohda
ZTF18aarohda
0.1385719327137984
ZTF18aabpxaf
LC good
ZTF18aabpxaf
LC good
ZTF18aagtevs
LC good
ZTF18aagtevs
LC good
ZTF18aailvrv
LC good
ZTF18aailvrv
LC good
ZTF18aalsomy
LC good
ZTF18aalsomy
ZTF18aaqculk
LC good
ZTF18aaqculk
LC good
ZTF18aarijdh
ZTF18aarijdh
LC good
ZTF18aarqjgu
ZTF18aarqjgu
ZTF18aatwwqq
0.1386895659334196
ZTF18aapcbys
LC good
ZTF18aapcbys
LC good
ZTF18aaqffy

ZTF18aaqgbju
ZTF18aaxmnqg
0.1544524173626632
ZTF18aahhbkj
LC good
ZTF18aaqcuba
ZTF18aaqcuba
ZTF18aaqnawz
LC good
ZTF18aaqnawz
LC good
ZTF18aarogfn
LC good
ZTF18aarogfn
ZTF18aarqjgu
ZTF18aarqjgu
ZTF18aataafd
ZTF18aataafd
ZTF18aautrkr
LC good
0.15457005058228443
0.15468768380190565
ZTF18aarlbnr
LC good
ZTF18aaroihe
ZTF18aaroihe
0.1548053170215269
0.1549229502411481
0.15504058346076932
0.15515821668039054
ZTF18aajpjdi
LC good
ZTF18aaqedfj
ZTF18aaxnoxv
ZTF18aaxnoxx
LC good
ZTF18aaxnoxx
0.15527584990001175
0.155393483119633
0.1555111163392542


Let's count the number of sources with good Caltech 1DC light curves.

In [167]:
files = glob.glob("output/*duration.txt")
src = []
for f in files:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        [src.append(val) for val in np.loadtxt(f, dtype=str, ndmin=2)]
src = np.unique(np.array(src))

In [168]:
print(src)

['ZTF17aabworn' 'ZTF18aaaqhra' 'ZTF18aaavliz' 'ZTF18aaavsda'
 'ZTF18aabejzt' 'ZTF18aabexlf' 'ZTF18aablarq' 'ZTF18aabtahy'
 'ZTF18aaccvcu' 'ZTF18aaelphu' 'ZTF18aagrdcs' 'ZTF18aagrdur'
 'ZTF18aagrhnw' 'ZTF18aagrhpx' 'ZTF18aagrtxs' 'ZTF18aagsegy'
 'ZTF18aagstdc' 'ZTF18aaguhgb' 'ZTF18aahaymm' 'ZTF18aahfeiy'
 'ZTF18aahfemn' 'ZTF18aahghck' 'ZTF18aahijwe' 'ZTF18aahiqfi'
 'ZTF18aahiqfv' 'ZTF18aahiqja' 'ZTF18aahiqyo' 'ZTF18aahirrj'
 'ZTF18aahjdxh' 'ZTF18aahjmat' 'ZTF18aahpyxj' 'ZTF18aahpzhc'
 'ZTF18aahqavd' 'ZTF18aahqmsr' 'ZTF18aahsgwp' 'ZTF18aahxqzi'
 'ZTF18aahzwvx' 'ZTF18aaigpfu' 'ZTF18aaiipqt' 'ZTF18aaikcbb'
 'ZTF18aaiwkwk' 'ZTF18aajopin' 'ZTF18aajpsyr' 'ZTF18aajthgd'
 'ZTF18aakjmwk' 'ZTF18aakoylt' 'ZTF18aalbpll' 'ZTF18aalrxas'
 'ZTF18abvkwla']


In [169]:
len(src)

49